In [1]:
import pandas as pd
import pathlib
import os
import re

# 1) Get all participants from respective datasets

In [14]:
# for both sheets in excel
df = pd.read_excel("english-results-data.xlsx", sheet_name="Time 1")
df1 = pd.read_excel("english-results-data.xlsx", sheet_name="Repeats")

C:\Users\imruh\miniconda3\envs\thesis\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\imruh\miniconda3\envs\thesis\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
print(df["WAB Type"].unique())
print(df1["WAB Type"].unique())

[nan 'Broca' 'TransMotor' 'not aphasic' 'Conduction' 'Wernicke' 'Anomic'
 'TransMotor ' 'U' 'Anomic ' 'Global' 'TransSensory']
[nan 'Broca' 'Conduction' 'Anomic' 'Wernicke' 'Global' 'TransMotor' 'U'
 'not aphasic' 'TransMotor ' "Broca's" 'not aphasic  ']


In [4]:
def get_participants_files(wab_type, data1, data2):
    data1 = data1.loc[data1["WAB Type"] == wab_type]
    participants = set(data1["Participant ID"])
    data2 = data2.loc[data2["WAB Type"] == wab_type]
    participants1 = set(data2["Participant ID"])
    
    participants.update(participants1)
    ids = list(participants)
    ids.sort(key=str.lower)
    return ids

In [5]:
# number of brocas patients
ids = get_participants_files("Broca", df, df1)
len(ids)

326

# 2) Get file names we are interested in for PWA

In [58]:
def get_file_names(wab_type, data1, data2):
    ids = get_participants_files(wab_type, data1, data2)
    brocas = [] # to check for missing data
    filenames = [] # to keep track of file names
    
    lower_ids = [x.lower() for x in ids]
    
    data_dir = str(pathlib.Path().resolve()) + "\\data_broca_1\\"
    
    for subdir, dirs, files in os.walk(data_dir):
        for file in files:
            i = file.replace(".cha","")
            dirname = os.path.basename(subdir).lower()
            # each naming convention is different
            if dirname == "fridriksson-2" and dirname + '-' + i in lower_ids:
                brocas.append(dirname + '-' + i)
                filenames.append(file)
            if i.lower() in lower_ids:
                brocas.append(i)
                filenames.append(file)
            if dirname+i.lower() in lower_ids:
                brocas.append(dirname+i)
                filenames.append(file)
    print(f"Found datafiles: {len(brocas)}")
    print(f"Actual datafiles in excel: {len(ids)}")
    
    lower_brocas= [x.lower() for x in brocas]
    
    print(f"Missing datafiles: {set(lower_ids) - set(lower_brocas)}")
    # missing data
    print(f"Number of missing datafiles: {len(set(lower_ids) - set(lower_brocas))}")
    
    return filenames

In [59]:
WAB = "Broca" 
fname = "broca_fname.csv"

filenames = get_file_names(WAB,df, df1)

df = pd.DataFrame(filenames).T
df.to_csv(fname, index=False, header=False)

Found datafiles: 164
Actual datafiles in excel: 326
Missing datafiles: {'fridriksson-2-1096-6', 'fridriksson-2-1015-3', 'fridriksson-2-1080-6', 'fridriksson-2-1015-5', 'fridriksson-2-1010-2', 'fridriksson-2-1064-5', 'fridriksson-2-1015-1', 'fridriksson-2-1082-5', 'fridriksson-2-1053-1', 'fridriksson-2-1080-3', 'fridriksson-2-1072-5', 'fridriksson-2-1036-4', 'fridriksson-2-1050-1', 'fridriksson-2-1012-4', 'umd-mma18a', 'fridriksson-2-1102-larc', 'fridriksson-2-1016-larc', 'fridriksson-2-1096-1', 'fridriksson-2-1064-2', 'fridriksson-2-1062-5', 'fridriksson-2-1096-larc', 'fridriksson-2-1095-larc', 'fridriksson-2-1062-larc', 'fridriksson-2-1064-larc', 'fridriksson-2-1064-1', 'umd-mma08a', 'fridriksson-2-1102-4', 'fridriksson-2-1036-6', 'fridriksson-2-1063-3', 'fridriksson-2-1015-larc', 'fridriksson-2-1106-5', 'fridriksson-2-1008-larc', 'fridriksson-2-1064-4', 'umd-mma14a', 'fridriksson-2-1050-3', 'fridriksson-2-1080-larc', 'fridriksson-2-1016-3', 'fridriksson-2-1102-6', 'fridriksson-2-1067

# 3) Getting the data preprocessed
#### Run setup_dataframe.py and then preprocessing_edited.py

# 4) Section the preprocessed data into scenarios (merge sentences) and combine broca data and control data

In [31]:
broca_fname = "data_broca_preprocessed.csv"
control_fname = "data_control_preprocessed.csv"

broca_data = pd.read_csv(broca_fname).dropna()
control_data = pd.read_csv(control_fname, encoding='utf8').dropna()

In [32]:
broca_patients = len(broca_data["source_file"].unique())
control = len(control_data["source_file"].unique())
print(f"Brocas patients: {broca_patients}, control: {control}")

Brocas patients: 164, control: 346


In [33]:
def merge_scenario(data, label):
    columns = ['source_file', 'scenario', 'text', 'label']
    new_data = [] # pd.DataFrame(columns=columns)
    text = ""

    for index, row in data.iterrows():
        # print(f"index : {index}, scenario {row['scenario']}")
        if index == 0:
            prev_scenario = row["scenario"] 
            prev_sourcefile = row["source_file"]
        current_scenario = row["scenario"]
        current_sourcefile = row["source_file"]
        
        if current_scenario == prev_scenario and current_sourcefile == prev_sourcefile:
            # print("same scenario")
            # print(text)
            # print()
            text += " " + row["preprocessed_text"]

        else:
            # print("new scenario")
            # print(text)
            # print()
            new_data.append([row["source_file"], prev_scenario, re.sub(' +', ' ', text), label])
            text = ""
        
        prev_scenario = current_scenario
        prev_sourcefile = current_sourcefile
        
        # if index == 74:
        #     break
    
    return pd.DataFrame(new_data, columns=columns)

In [7]:
# LABEL 1 FOR BROCA 0 FOR NOT APHASIC, for classifier
broca_data_merge = merge_scenario(broca_data, 1)
control_data_merge = merge_scenario(control_data, 0)
data_full_scenario = pd.concat([broca_data_merge, control_data_merge], ignore_index=True)

In [8]:
# A bit imbalanced?
print(len(broca_data_merge))
print(len(control_data_merge))

1155
2076


In [9]:
# shuffle data
data_full_scenario = data_full_scenario.sample(frac=1).reset_index(drop=True)

In [10]:
data_full_scenario

,source_file,scenario,text,label
0,1030-1.cha,Cinderella,aw man I tea. okay fine then he wanted to doe...,1
1,fridriksson12a.cha,Cinderella,I saw it. and there is two girls and the othe...,1
2,kurland19c.cha,Cinderella,oh oh Cinderella good night sweet night midni...,1
3,wright207a.cha,Umbrella,rain. yeah. snow rain rain rain rain. wow. ra...,1
4,richardson204.cha,Cat,there is a lot of things going on here. so th...,0
...,...,...,...,...
3226,cmu02b.cha,Window,alright. alright. kicked it. whoops. wntho wn...,1
3227,capilouto21a.cha,Sandwich,if I were to do it I would start by opening u...,0
3228,cmu02b.cha,Cat,cat. I want it. up the hrid. leaping. cat. I ...,1
3229,tucson02a.cha,Window,okay. basketball. and he is very good. stk. g...,1


In [112]:
data_full_scenario.to_csv("all_data_processed.csv")

# 5) Alternative: don't merge scenarios

In [2]:
broca_fname = "test1.csv"
control_fname = "test2.csv"

broca_data = pd.read_csv(broca_fname).dropna()
control_data = pd.read_csv(control_fname, encoding='utf8').dropna()

broca_data["label"] = 1
control_data["label"] = 0

data_full_scenario = pd.concat([broca_data, control_data], ignore_index=True)
data_full_scenario = data_full_scenario.sample(frac=1).reset_index(drop=True)
data_full_scenario["preprocessed_text"] = [x.lstrip().rstrip().lower() for x in data_full_scenario['preprocessed_text']]

data_full_scenario.to_csv("all_sentences_processed.csv")

In [3]:
data_full_scenario.to_csv("input.csv")

In [5]:
data_full_scenario.preprocessed_text.to_csv("test.csv")